# Demo - Performance Comparison (CIFAR10)

## 1. Load CIAFR10

In [ ]:
import sys
sys.path.insert(0, '/home/aditya/AdversarialAttacks')

In [ ]:
import timm

from src.data import load_cifar10, load_cifar10c
from src.utils import load_model, clean_accuracy
from utils import l2_distance

images, labels = load_cifar10(n_examples=1000)
# imagesc, labelsc = load_cifar10c(n_examples=1000)
device = "cuda"

In [ ]:
model_list = ['vgg19', 'inception_v3', 'resnetv2_152', 'resnest50d', 'wide_resnet101_2', 'resnext101_64x4d', 
              'vit_base_patch8_224', 'swinv2_cr_base_224', 'deit3_base_patch16_224', 'xcit_large_24_p8_224']

## 2. Standard Accuracy

In [ ]:
for model_name in model_list:
    model = load_model(model_name).to(device)
    acc = clean_accuracy(model, images.to(device), labels.to(device))
    print('Model: {}'.format(model_name))
    print('- Standard Acc: {}'.format(acc))

## 3. Attacks on the models

In [ ]:
import datetime
import numpy as np
import warnings

warnings.filterwarnings(action='ignore')

import torch
import torch.nn as nn
import torch.optim as optim

import attacks

### FGSM

In [ ]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name).to(device)
    
    print("- Torchattacks")
    atk = attacks.FGSM(model, eps=8/255)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

### BIM

In [ ]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name).to(device)
    
    print("Attacks")
    atk = attacks.BIM(model, eps=8/255, alpha=2/255, steps=10)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

### PGD

In [ ]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name).to(device)
    
    print("- Torchattacks")
    atk = attacks.PGD(model, eps=8/255, alpha=2/255, steps=10, random_start=False)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    print('- Robust Acc: {} ({} ms)'.format(acc, int((end-start).total_seconds()*1000)))
    
    print()

## DeepFool

In [ ]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = attacks.DeepFool(model, steps=50, overshoot=0.02)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

### CW

In [ ]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name, norm='Linf').to(device)
    
    print("- Torchattacks")
    atk = attacks.CW(model, c=1, kappa=0, steps=100, lr=0.01)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

### DeepFool

In [ ]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name).to(device)
    
    print("- Torchattacks")
    atk = attacks.DeepFool(model)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

### AutoAttack

In [ ]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name).cuda()
    
    print("- Torchattacks")
    atk = attacks.AutoAttack(model)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

### Square

In [ ]:
for model_name in model_list:
    print('Model: {}'.format(model_name))
    model = load_model(model_name).cuda()
    
    print("- Torchattacks")
    atk = attacks.Square(model)
    start = datetime.datetime.now()
    adv_images = atk(images, labels)
    end = datetime.datetime.now()
    acc = clean_accuracy(model, adv_images, labels)
    l2 = l2_distance(model, images, adv_images, labels, device=device)
    print('- Robust Acc: {} ({} ms)'.format(acc, l2,
                                                         int((end-start).total_seconds()*1000)))
    
    print()

In [ ]:
print(timm.__version__)